In [10]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

import gymnasium as gym
import numpy as np
from keras.models import Sequential
from keras.layers import InputLayer
from keras.layers import Dense

import matplotlib.pyplot as plt

In [11]:
for i in range(num_episodes):
    state = env.reset()
    eps *= eps_decay_factor
    done = False
    while not done:
        if np.random.random() < eps or np.sum(q_table[state, :]) == 0:
            action = np.random.randint(0, env.action_space.n)
        else:
            action = np.argmax(q_table[state, :])
        new_state, reward, done, _ = env.step(action)
        q_table[state, action] += 
          reward + 
            learning_rate * 
              (discount_factor * np.max(q_table[new_state, :]) - q_table[state, action])
        state = new_state
        
for i in range(num_episodes):
    state = env.reset()
    eps *= eps_decay_factor
    done = False
    while not done:
        if np.random.random() < eps:
            action = np.random.randint(0, env.action_space.n)
        else:
            action = np.argmax(
              model.predict(np.identity(env.observation_space.n)[state:state + 1]))
        new_state, reward, done, _ = env.step(action)
        target = reward + 
          discount_factor * 
            np.max(
              model.predict(
                np.identity(env.observation_space.n)[new_state:new_state + 1]))
        target_vector = model.predict(
          np.identity(env.observation_space.n)[state:state + 1])[0]
        target_vector[action] = target
        model.fit(
          np.identity(env.observation_space.n)[state:state + 1], 
          target_vec.reshape(-1, env.action_space.n), 
          epochs=1, verbose=0)
        state = new_state

SyntaxError: invalid syntax (3298449353.py, line 11)

In [65]:
import random

class Agent:
    def __init__(self, states, actions, epsilon=0.7, gamma=0.7, epsilon_decay=0.9):
        self.actions = actions
        self.states = states
        self.epsilon = epsilon
        self.gamma = gamma
        self.epsilon_decay = epsilon_decay
        
        self.model = Sequential()
        self.model.add(InputLayer(batch_input_shape=(1, env.observation_space.n)))
        self.model.add(Dense(20, activation='relu'))
        self.model.add(Dense(env.action_space.n, activation='linear'))
        self.model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    
    def act(self, state):
        if random.random() < self.epsilon:
            action = random.randint(0, env.action_space.n)
        else:
            action = np.argmax(self.model.predict(np.identity(env.observation_space.n)[state[0]:state[0]+1]))
            self.epsilon = self.epsilon * self.epsilon_decay
            
        return action

    def update(self, state, action, reward, next_state):
        new_q = reward + self.gamma*np.max(self.model.predict(np.identity(env.observation_space.n)[next_state:next_state+1]))
        next_state_vec = self.model.predict(np.identity(env.observation_space.n)[state[0]:state[0]+1][0])
        next_state_vec[action] = new_q
        self.model.fit(np.identity(env.observation_space.n)[state[0]:state[0]+1], next_state_vec.reshape(-1, self.actions), epochs=1)

In [67]:
import gymnasium as gym
#import agents

env = gym.make('Taxi-v3')
agent = Agent(states=env.observation_space.n, actions=env.action_space.n)

episodes = 10000
total_steps = 0
total_reward = 0

for episode in range(episodes):
    
    state = env.reset()
    reward = 0
    steps = 0
    done = False
    
    while not done:
        action = agent.act(state) 
        next_state, reward, done, truncated, _ = env.step(action)
        agent.update(state, action, reward, next_state)
        state = next_state
        
        steps += 1
        total_reward += reward

    total_steps += steps
    
print("Average timesteps taken: {}".format(total_steps/episodes))
print("Average reward: {}".format(total_reward/episodes))
print("Total reward: {}".format(total_reward))

1/1 [==============================] - 0s 13ms/step


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_14/dense_26/MatMul' defined at (most recent call last):
    File "C:\Users\Nazia.Habib\Anaconda3\envs\dsa\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Nazia.Habib\Anaconda3\envs\dsa\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\Nazia.Habib\Anaconda3\envs\dsa\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Nazia.Habib\Anaconda3\envs\dsa\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\Nazia.Habib\Anaconda3\envs\dsa\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\Nazia.Habib\Anaconda3\envs\dsa\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\User

In [58]:
model.predict(np.identity(500)[3:4])

1/1 [==============================] - 0s 26ms/step


array([[-0.02726975, -0.04312754,  0.09129141, -0.01532543,  0.00911291,
         0.02122563]], dtype=float32)

In [55]:
model = Sequential()
model.add(InputLayer(batch_input_shape=(1, env.observation_space.n)))
model.add(Dense(20, activation='relu'))
model.add(Dense(env.action_space.n, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [59]:
state = env.reset()

In [60]:
state

(366, {'prob': 1.0, 'action_mask': array([1, 1, 1, 0, 0, 0], dtype=int8)})